## Keras Modell Konvertieren
Für die Nutzung des trainierten Keras Modells auf anderen Hardware-Plattformen ist es sinnvoll, dieses in das Open Neural Network Exchange (ONNX) Format zu konvertieren. 

### Zugriff auf Cloud Speicher
Um die Modell-Daten Notebook-übergreifend zu speichern, wird Google Drive genutzt, welches wie folgt innerhalb von Google Colab gemountet werden kann:

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Modell-Pfad
Die trainierten tf.keras-Modelle liegen in folgendem Verzeichnis:

In [22]:
%cd /content/drive/My\ Drive/data/ix-tut-model/
!ls -ls

/content/drive/My Drive/data/ix-tut-model
total 240878
     4 drwx------ 3 root root      4096 Apr 13 10:57 old
    58 -rw------- 1 root root     59145 Apr 13 11:47 starwars_yolov3.data-00000-of-00002
240617 -rw------- 1 root root 246391528 Apr 13 11:47 starwars_yolov3.data-00001-of-00002
    24 -rw------- 1 root root     24271 Apr 13 11:47 starwars_yolov3.index
   175 -rw------- 1 root root    178754 Apr 13 11:47 starwars_yolov3.json


### Installation Tensorflow
An dieser Stelle wird Tensorflow in der älteren Version 1.15.2 genutzt, da der Keras2ONNX-Konverter noch Probleme mit der Modellkonvertierung mit Tensorfor 2.x hat.

In [3]:
!pip install tensorflow==1.15.2

     |████████████████████████████████| 110.5MB 78kB/s 
     |████████████████████████████████| 3.8MB 43.6MB/s 
     |████████████████████████████████| 512kB 41.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=dfd44d3033fa25710c49a1f354c2ac5b96532139cd72a2c42ce3ccf3b85f5cb3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

### Installation keras2onnx
Im Folgenden werden die nötigen Bibliotheken installieren um das Keras Modell in ein ONNX-Modell zu konvertieren.

In [4]:
!pip install git+https://github.com/microsoft/onnxconverter-common
!pip install git+https://github.com/onnx/keras-onnx

  Cloning https://github.com/microsoft/onnxconverter-common to /tmp/pip-req-build-rpjmhymc
  Running command git clone -q https://github.com/microsoft/onnxconverter-common /tmp/pip-req-build-rpjmhymc
     |████████████████████████████████| 4.8MB 3.4MB/s 
  Created wheel for onnxconverter-common: filename=onnxconverter_common-1.7.0-py2.py3-none-any.whl size=55213 sha256=93dd6bf7f00ff8fc9048d0a48b062b73876825f10b0912dc78213a97b7e2c935
  Stored in directory: /tmp/pip-ephem-wheel-cache-ymz0gpwp/wheels/7d/97/c7/8908efff309db78064933e54f2a70427a75ada73b9b19789c3
Successfully built onnxconverter-common
  Cloning https://github.com/onnx/keras-onnx to /tmp/pip-req-build-6yi3ww83
  Running command git clone -q https://github.com/onnx/keras-onnx /tmp/pip-req-build-6yi3ww83
     |████████████████████████████████| 81kB 2.6MB/s 
  Created wheel for keras2onnx: filename=keras2onnx-1.7.0-cp36-none-any.whl size=91310 sha256=f2ed052c9d7205d7c3406d859831f96565e21b36888c230ed43f6cea894e5c2c
  Stored in di

### Tutorial Quellcode installieren
Der Quellcode aus den vorangegangenen Tutorials wird benötigt um das Keras Modell zu erzeugen.

In [7]:
%cd /content/
!git clone https://github.com/rawar/ix-tut-yolov3.git

/content
Cloning into 'ix-tut-yolov3'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 87 (delta 34), reused 73 (delta 22), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [11]:
%cd /content/ix-tut-yolov3
!git clone https://github.com/rawar/ix-tut-yolov3-data.git

/content/ix-tut-yolov3
Cloning into 'ix-tut-yolov3-data'...
remote: Enumerating objects: 2672, done.
remote: Total 2672 (delta 0), reused 0 (delta 0), pack-reused 2672
Receiving objects: 100% (2672/2672), 52.62 MiB | 34.63 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [12]:
%cd /content/ix-tut-yolov3

/content/ix-tut-yolov3


### Keras nach ONNX
Der folgende Python-Code konvertiert das bestehende Keras-Modell in das Open Neuronal Network Exchange Format.


#### Referenzierung der nötigen Module
Neben Tensorflow, und dem YOLOv3-Modell werden die Module onnx und keras2onnx referenziert.

In [0]:
import os
os.environ['TF_KERAS'] = '1'
import onnx
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model
from core.yolov3 import YOLOv3, decode
import onnx
import keras2onnx

#### Modell-Dateinamen definiere
Die folgenden Dateinamen referenzieren auf die Modell-Datei und legen den Namen für die ONNX-Datei fest.

In [0]:
keras_model_filename = "/content/drive/My Drive/data/ix-tut-model/starwars_yolov3"
onnx_model_filename = "/content/drive/My Drive/data/ix-tut-model/starwars_yolov3.onnx"

#### Modell aufbauen
Das Modell wird instantiiert und die trainierten Gewichte geladen.

In [15]:
input_size   = 416
input_layer  = tf.keras.layers.Input([input_size, input_size, 3])
feature_maps = YOLOv3(input_layer)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
bbox_tensors = []
for i, fm in enumerate(feature_maps):
    bbox_tensor = decode(fm, i)
    bbox_tensors.append(bbox_tensor)

keras_model = tf.keras.Model(input_layer, bbox_tensors)

In [25]:
keras_model.load_weights(keras_model_filename)

#### Modell konvertieren
Die folgenden beiden Zeilen führen die eigentliche Konvertierung durch und speichern das trainierte Modell im ONNX-Format.

In [26]:
onnx_model = keras2onnx.convert_keras(keras_model,keras_model.name)

The node number after optimization: 1497 -> 322


In [0]:
onnx.save_model(onnx_model, onnx_model_filename)

#### ONNX-Modell
Das so erzeugte ONNX-Modell liegt auf dem GDrive neben dem bestehenden Keras-Modell.

In [28]:
%cd /content/drive/My\ Drive/data/ix-tut-model/
!ls -ls

/content/drive/My Drive/data/ix-tut-model
total 481240
     4 drwx------ 3 root root      4096 Apr 13 10:57 old
    58 -rw------- 1 root root     59145 Apr 13 11:47 starwars_yolov3.data-00000-of-00002
240617 -rw------- 1 root root 246391528 Apr 13 11:47 starwars_yolov3.data-00001-of-00002
    24 -rw------- 1 root root     24271 Apr 13 11:47 starwars_yolov3.index
   175 -rw------- 1 root root    178754 Apr 13 11:47 starwars_yolov3.json
240362 -rw------- 1 root root 246130069 Apr 13 12:06 starwars_yolov3.onnx


#### ONNX-Tools
Die ONNX-Bibliothek enthält noch weitere Tools für die Modell-Verarbeitung. Mit dem 'net_drawer.py' Skript lassen sich bestehende ONNX-Modell mit Hilfe der GraphViz-Bibliothek visualisieren.

In [0]:
!python3 /usr/local/lib/python3.6/dist-packages/onnx/tools/net_drawer.py --input '/content/drive/My Drive/data/ix-tut-model/starwars_yolov3.onnx' --output '/content/drive/My Drive/data/ix-tut-model/starwars_yolov3.dot' --embed_docstring

In [0]:
!dot -Tsvg '/content/drive/My Drive/data/ix-tut-model/starwars_yolov3.dot' -o '/content/drive/My Drive/data/ix-tut-model/starwars_yolov3.svg' 